In [1]:
import psycopg2

In [14]:
# Connect to the postgres database
conn = psycopg2.connect("host=d2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com dbname=d2b_assessment user=peluader5437 password=QjtB6Djwd0")
cur = conn.cursor()

In [189]:
cur.execute("SELECT product_name, SUM(review) as tt_review FROM if_common.dim_products dp JOIN peluader5437_staging.reviews sr ON dp.product_id = sr.product_id GROUP BY product_name;")

# conn.commit()

In [190]:
data = cur.fetchall()

In [197]:
import pandas as pd
cols = []
for elt in cur.description:
    cols.append(elt[0])
df = pd.DataFrame(data=data, columns=cols)
df.to_csv('./dataset/test.csv')

In [198]:
conn.close()

In [98]:
cur.execute("SELECT MAX(tt_review) FROM (SELECT product_name, SUM(review) as tt_review FROM if_common.dim_products dp JOIN peluader5437_staging.reviews sr ON dp.product_id = sr.product_id GROUP BY product_name) AS tt;") # Sql command to list all available schemas in the database
conn.commit()

In [99]:
cur.fetchall()

[(967,)]

In [21]:
cur.execute("SELECT schema_name FROM information_schema.schemata;") # Sql command to list all available schemas in the database
conn.commit()


In [22]:
cur.fetchall() #List all available schemas

[('pg_catalog',),
 ('information_schema',),
 ('public',),
 ('if_common',),
 ('peluader5437_staging',),
 ('peluader5437_analytics',)]

In [25]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'peluader5437_staging' ORDER BY table_name;") #SQL command to print the tables in this particular schema
conn.commit() 

In [26]:
cur.fetchall() # From the result, we can see there is no table yet in this schema

[('orders',), ('reviews',), ('shipments_deliveries',)]

In [29]:
cur.execute("SELECT * FROM information_schema.tables WHERE table_schema = 'peluader5437_analytics';") #SQL command to print the tables in this particular schema
conn.commit()

In [30]:
cur.fetchall() # From the result, we can see there is no table yet in this schema

[]

In [108]:
cur.execute("select * from peluader5437_staging.orders LIMIT 50;") # SQL command to print the tables in this particular schema
conn.commit()

In [109]:
cur.fetchall() # We can see all the tables in the if_common schema listed

[(1, 5, datetime.date(2022, 7, 13), '24', 139, 10, 1390),
 (2, 14, datetime.date(2021, 4, 6), '2', 273, 4, 1092),
 (3, 17, datetime.date(2022, 7, 29), '20', 253, 9, 2277),
 (4, 14, datetime.date(2022, 8, 27), '8', 334, 1, 334),
 (5, 25, datetime.date(2021, 12, 15), '6', 334, 3, 1002),
 (6, 7, datetime.date(2021, 4, 15), '25', 144, 7, 1008),
 (7, 4, datetime.date(2021, 9, 28), '12', 876, 1, 876),
 (8, 24, datetime.date(2021, 5, 8), '15', 994, 7, 6958),
 (9, 3, datetime.date(2022, 1, 6), '18', 641, 4, 2564),
 (10, 23, datetime.date(2021, 8, 22), '3', 841, 5, 4205),
 (11, 3, datetime.date(2022, 1, 25), '8', 334, 8, 2672),
 (12, 21, datetime.date(2022, 1, 15), '12', 413, 4, 1652),
 (13, 22, datetime.date(2021, 6, 25), '9', 596, 4, 2384),
 (14, 6, datetime.date(2021, 10, 4), '15', 997, 9, 8973),
 (15, 8, datetime.date(2021, 2, 27), '22', 284, 1, 284),
 (16, 11, datetime.date(2021, 3, 9), '13', 266, 10, 2660),
 (17, 23, datetime.date(2022, 5, 27), '11', 455, 8, 3640),
 (18, 5, datetime.date(

In [110]:
cur.execute("select * from if_common.dim_dates;") # SQL command to print the tables in this particular schema
conn.commit()

In [111]:
cur.fetchall() # We can see all the tables in the if_common schema listed

[(datetime.date(2015, 1, 1), 2015, 1, 1, 4, False),
 (datetime.date(2015, 1, 2), 2015, 1, 2, 5, True),
 (datetime.date(2015, 1, 3), 2015, 1, 3, 6, False),
 (datetime.date(2015, 1, 4), 2015, 1, 4, 7, False),
 (datetime.date(2015, 1, 5), 2015, 1, 5, 1, True),
 (datetime.date(2015, 1, 6), 2015, 1, 6, 2, True),
 (datetime.date(2015, 1, 7), 2015, 1, 7, 3, True),
 (datetime.date(2015, 1, 8), 2015, 1, 8, 4, True),
 (datetime.date(2015, 1, 9), 2015, 1, 9, 5, True),
 (datetime.date(2015, 1, 10), 2015, 1, 10, 6, False),
 (datetime.date(2015, 1, 11), 2015, 1, 11, 7, False),
 (datetime.date(2015, 1, 12), 2015, 1, 12, 1, True),
 (datetime.date(2015, 1, 13), 2015, 1, 13, 2, True),
 (datetime.date(2015, 1, 14), 2015, 1, 14, 3, True),
 (datetime.date(2015, 1, 15), 2015, 1, 15, 4, True),
 (datetime.date(2015, 1, 16), 2015, 1, 16, 5, True),
 (datetime.date(2015, 1, 17), 2015, 1, 17, 6, False),
 (datetime.date(2015, 1, 18), 2015, 1, 18, 7, False),
 (datetime.date(2015, 1, 19), 2015, 1, 19, 1, True),
 (da

In [93]:
conn.close()

In [153]:
agg_public_holiday_table_create = (""" 
     CREATE TABLE peluader5437_analytics.agg_public_holiday (
            ingestion_date date NOT NULL PRIMARY KEY,
            tt_order_hol_jan INT NOT NULL,
            tt_order_hol_feb INT NOT NULL,
            tt_order_hol_mar INT NOT NULL,
            tt_order_hol_apr INT NOT NULL,
            tt_order_hol_may INT NOT NULL,
            tt_order_hol_jun INT NOT NULL,
            tt_order_hol_jul INT NOT NULL,
            tt_order_hol_aug INT NOT NULL,
            tt_order_hol_sep INT NOT NULL,
            tt_order_hol_oct INT NOT NULL,
            tt_order_hol_nov INT NOT NULL,
            tt_order_hol_dec INT NOT NULL
        );
""")

In [154]:
agg_public_holiday_table_insert = '''INSERT INTO peluader5437_analytics.agg_public_holiday (
            ingestion_date, tt_order_hol_jan, tt_order_hol_feb, tt_order_hol_mar,
            tt_order_hol_apr, tt_order_hol_may, tt_order_hol_jun, tt_order_hol_jul,
            tt_order_hol_aug, tt_order_hol_sep, tt_order_hol_oct, tt_order_hol_nov,
            tt_order_hol_dec)
            SELECT
            now() AS ingestion_date,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2  
            WHERE EXTRACT(MONTH FROM t2.order_date) = 1) AS tt_order_hol_jan,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 2) AS tt_order_hol_feb,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 3) AS tt_order_hol_mar,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 4) AS tt_order_hol_apr,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 5) AS tt_order_hol_may,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 6) AS tt_order_hol_jun,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 7) AS tt_order_hol_jul,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 8) AS tt_order_hol_aug,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 9) AS tt_order_hol_sep,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 10) AS tt_order_hol_oct,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 11) AS tt_order_hol_nov,
            (SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 12) AS tt_order_hol_dec
            
            '''

In [155]:
agg_public_holiday_table_drop = "drop table if exists peluader5437_analytics.agg_public_holiday"

cur.execute(agg_public_holiday_table_drop)
conn.commit()

In [156]:
cur.execute(agg_public_holiday_table_create)
conn.commit()

In [157]:
cur.execute(agg_public_holiday_table_insert)
conn.commit()

In [4]:
cur.execute('''SELECT
            SUM(t2.quantity) FROM (select order_date, quantity  FROM peluader5437_staging.orders so 
            JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 
            WHERE EXTRACT(MONTH FROM t2.order_date) = 10;''') # SQL command to print the tables in this particular schema
conn.commit()

In [5]:
cur.fetchall()

[(96,)]

In [6]:
cur.execute('''SELECT * FROM (select *  FROM peluader5437_staging.orders so 
            JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2 ;''') # SQL command to print the tables in this particular schema
conn.commit()

In [7]:
cur.fetchall()

[(3528,
  3,
  datetime.date(2021, 1, 1),
  '20',
  438,
  2,
  876,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (2000,
  2,
  datetime.date(2021, 1, 1),
  '6',
  103,
  6,
  618,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (9321,
  6,
  datetime.date(2021, 1, 1),
  '18',
  552,
  9,
  4968,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (6109,
  9,
  datetime.date(2021, 1, 1),
  '11',
  359,
  9,
  3231,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (9883,
  1,
  datetime.date(2021, 1, 1),
  '23',
  50,
  6,
  300,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (2773,
  5,
  datetime.date(2021, 1, 1),
  '24',
  126,
  5,
  630,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (3372,
  8,
  datetime.date(2021, 1, 1),
  '19',
  487,
  6,
  2922,
  datetime.date(2021, 1, 1),
  2021,
  1,
  1,
  5,
  False),
 (2258,
  1,
  datetime.date(2021, 1, 1),
  '14',
  101,
  5,
  505,
  datetime.da

In [177]:
conn.close()

In [160]:
agg_shipments_table_create = (""" 
     CREATE TABLE peluader5437_analytics.agg_shipments (
            ingestion_date date NOT NULL PRIMARY KEY,
            tt_late_shipments INT NOT NULL,
            tt_undelivered_items INT NOT NULL
        );
""")


In [161]:
agg_shipments_table_drop = "drop table if exists peluader5437_analytics.agg_shipments"

In [162]:
cur.execute(agg_shipments_table_drop)
conn.commit()

In [163]:
cur.execute(agg_shipments_table_create)
conn.commit()

In [164]:
agg_shipments_table_insert = '''INSERT INTO peluader5437_analytics.agg_shipments (
            ingestion_date, tt_late_shipments, tt_undelivered_items)
            SELECT
            now() AS ingestion_date,
            (SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 WHERE DATE_PART('day', t1.shipment_date::timestamp - t1.order_date::timestamp) >= 6 AND t1.delivery_date IS NULL )  AS tt_late_shipments,
            
            (SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 WHERE t1.shipment_date IS NULL AND t1.delivery_date IS NULL AND DATE_PART('day', '2022-09-05'::timestamp - t1.order_date::timestamp) >= 15 )  AS tt_undelivered_items
            '''

In [165]:
cur.execute(agg_shipments_table_insert)
conn.commit()

In [131]:
cur.execute('''SELECT
            SUM(t2.quantity) FROM (select *  FROM peluader5437_staging.orders so 
            JOIN if_common.dim_dates idd ON so.order_date::timestamp = idd.calendar_dt::timestamp
            WHERE idd.day_of_the_week_num<6 AND idd.working_day=FALSE) t2  
            WHERE EXTRACT(MONTH FROM t2.order_date) = 3;''') # SQL command to print the tables in this particular schema
conn.commit()

In [132]:
cur.fetchall()

[(172,)]

In [178]:
best_performing_product_table_drop = "drop table if exists peluader5437_analytics.best_performing_product"

In [179]:
best_performing_product_table_create = (""" 
     CREATE TABLE peluader5437_analytics.best_performing_product (
            ingestion_date date NOT NULL PRIMARY KEY,
            product_name varchar NOT NULL,
            most_ordered_day date NOT NULL,
            is_public_holiday bool NOT NULL,
            tt_review_points INT NOT NULL,
            pct_one_star_review float NOT NULL,
            pct_two_star_review float NOT NULL,
            pct_three_star_review float NOT NULL,
            pct_four_star_review float NOT NULL,
            pct_five_star_review float NOT NULL,
            pct_early_shipments float NOT NULL,
            pct_late_shipments float NOT NULL
        );
""")

In [180]:
cur.execute(best_performing_product_table_drop)
conn.commit()

In [181]:
cur.execute(best_performing_product_table_create)
conn.commit()

In [ ]:
best_performing_product_table_insert = '''INSERT INTO peluader5437_analytics.best_performing_product (
            ingestion_date, product_name, most_ordered_day, is_public_holiday, tt_review_points,
            pct_one_star_review, pct_two_star_review, pct_three_star_review, pct_four_star_review,
            pct_five_star_review float, pct_early_shipments float, pct_late_shipments)
            SELECT
            now() AS ingestion_date,

            (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AS product_name,

            (SELECT t2.order_date FROM (SELECT * FROM if_common.dim_products dp JOIN peluader5437_staging.orders so ON dp.product_id = so.product_id::INT WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products dp ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties)) t2 
            GROUP BY order_date ORDER By SUM(t2.quantity) DESC fetch first 1 rows with ties) AS most_ordered_day,

            (SELECT
                CASE
                    WHEN (day_of_the_week_num < 6) AND (working_day = FALSE) THEN 'TRUE'
                    ELSE 'FALSE'
                END AS is_public_holiday
                FROM (SELECT t2.order_date FROM (SELECT * FROM if_common.dim_products dp JOIN peluader5437_staging.orders so ON dp.product_id = so.product_id::INT 
                WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp ON sr.product_id = dp.product_id 
                GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties)) t2 GROUP BY order_date ORDER By SUM(t2.quantity) DESC fetch first 1 rows with ties) t3
                JOIN if_common.dim_dates idd ON (t3.order_date::timestamp = idd.calendar_dt::timestamp)), 

            (SELECT MAX(tt_review) FROM (SELECT product_name, SUM(review) as tt_review FROM if_common.dim_products dp 
            JOIN peluader5437_staging.reviews sr ON dp.product_id = sr.product_id GROUP BY product_name) tt) AS tt_review_points,

            (SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
            ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 1)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_one_star_review,

            (SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
            ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 2)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_two_star_review,

            (SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
            ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 3)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_three_star_review,

            (SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
            ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 4)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_four_star_review,

            (SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
            ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 5)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
            JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_five_star_review,

            (SELECT (SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE DATE_PART('day', t1.shipment_date::timestamp - t1.order_date::timestamp) < 6 
            AND t1.delivery_date IS NOT NULL AND product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))/(SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100) AS pct_early_shipments,


            (SELECT (SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE DATE_PART('day', t1.shipment_date::timestamp - t1.order_date::timestamp) >= 6 
            AND t1.delivery_date IS NULL AND product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))/(SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 WHERE DATE_PART('day', t1.shipment_date::timestamp - t1.order_date::timestamp) >= 6 
            AND t1.delivery_date IS NULL)::float * 100) AS pct_late_shipments
            
            '''

In [149]:
conn.close()

In [216]:
cur.execute("SELECT t2.order_date FROM (SELECT * FROM if_common.dim_products dp JOIN peluader5437_staging.orders so ON dp.product_id = so.product_id::INT WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties)) t2 GROUP BY order_date ORDER By SUM(t2.quantity) DESC fetch first 1 rows with ties ;") # Sql command to list all available schemas in the database
conn.commit()

In [217]:
cur.fetchall()

[(datetime.date(2022, 1, 6),)]

In [125]:
cur.execute('''SELECT
CASE
    WHEN (day_of_the_week_num < 6) AND (working_day = FALSE) THEN 'TRUE'
    ELSE 'FALSE'
END AS is_public_holiday
FROM (SELECT t2.order_date FROM (SELECT * FROM if_common.dim_products dp JOIN peluader5437_staging.orders so ON dp.product_id = so.product_id::INT 
WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp ON sr.product_id = dp.product_id 
GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties)) t2 GROUP BY order_date ORDER By SUM(t2.quantity) DESC fetch first 1 rows with ties) t3
JOIN if_common.dim_dates idd ON (t3.order_date::timestamp = idd.calendar_dt::timestamp);

''')
conn.commit()

In [126]:
cur.fetchall()

[('FALSE',)]

In [30]:
cur.execute('''SELECT (SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE DATE_PART('day', t1.shipment_date::timestamp - t1.order_date::timestamp) >= 6 
            AND t1.delivery_date IS NULL AND product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))/(SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100;''')

In [32]:
cur.execute('''SELECT
            COUNT(*) FROM (SELECT *  FROM peluader5437_staging.orders so 
            JOIN peluader5437_staging.shipments_deliveries sd ON so.order_id = sd.order_id) t1 JOIN if_common.dim_products idp
            ON t1.product_id::INT = idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
            ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties);''')

In [33]:
cur.fetchall()

[(444,)]

In [13]:
cur.execute("SELECT SUM(review) FROM peluader5437_staging.reviews where review=2;")

In [14]:
cur.fetchall()

[(2844,)]

In [ ]:
conn.close()

In [15]:
cur.execute("SELECT SUM(review) FROM peluader5437_staging.reviews where review=3;")

In [16]:
cur.fetchall()

[(4209,)]

In [117]:
cur.execute("SELECT (SELECT COUNT(*) FROM peluader5437_staging.reviews where review=5)/(SELECT COUNT(*) FROM peluader5437_staging.reviews)::float;")

In [105]:
cur.execute("SELECT 53/309::float;")

In [118]:

cur.fetchall()

[(0.20301271420674405,)]

In [115]:
cur.execute("SELECT COUNT(review) FROM peluader5437_staging.reviews where review=5;")

In [116]:
cur.fetchall()

[(1469,)]

In [121]:
cur.execute('''SELECT  (SELECT COUNT(*) FROM peluader5437_staging.reviews sr JOIN if_common.dim_products idp
 ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties) AND review = 1)/(SELECT COUNT(*) FROM peluader5437_staging.reviews sr 
JOIN if_common.dim_products idp ON sr.product_id=idp.product_id WHERE product_name = (SELECT product_name FROM peluader5437_staging.reviews sr JOIN if_common.dim_products dp 
ON sr.product_id = dp.product_id GROUP BY product_name ORDER BY SUM(review) DESC fetch first 1 rows with ties))::float * 100;''')

In [122]:
cur.fetchall()

[(17.15210355987055,)]

In [25]:
cur.execute("SELECT product_name, SUM(review) as tt_review FROM if_common.dim_products dp JOIN peluader5437_staging.reviews sr ON dp.product_id = sr.product_id GROUP BY product_name;")

In [26]:
cur.fetchall()

[('Screwdriver', 940),
 ('Car', 821),
 ('Smartphone', 785),
 ('Jeans', 893),
 ('Underwear', 804),
 ('Camera', 868),
 ('Saucepan', 916),
 ('Hammer', 876),
 ('Wardrobe', 864),
 ('Bicycle', 912),
 ('Non-stick pan', 898),
 ('Armchair', 843),
 ('Tablet', 949),
 ('Scooter', 967),
 ('T-shirt', 842),
 ('Nails', 925),
 ('Bed', 898),
 ('Frying pan', 849),
 ('Laptop', 806),
 ('Shoes', 896),
 ('Motorcycle', 880),
 ('Sofa', 824),
 ('Books', 848),
 ('Jacket', 875),
 ('Socks', 834)]

In [ ]:
cur.execute("select * from peluader5437_analytics.best_performing_product;") # SQL command to print the tables in this particular schema
conn.commit()

In [ ]:
cur.fetchall()